# CLASS Performance Testing

Various tests of CLASS computation speed. No need to have inviz installed, just Classy and NumPy. Feel free to run these on your machine and add your results below!

In [ ]:
import numpy as np
from classy import Class
from multiprocessing import Pool

In [ ]:
test_params = {
    'omega_b': 2.268118e-2, 
    'omega_dmeff': 0.1221163, 
    'ln10^{10}A_s': 3.048006, 
    'n_s': 0.9670508, 
    'tau_reio': 0.05659527, 
    'sigma_dmeff': 1.193e-25, 
    'Omega_Lambda': 0.6772064, 
    'YHe': 0.2479716, 
    'h': 67.13298e-2,  
    'omega_cdm': 1e-15, 
    'npow_dmeff': 0, 
    'Vrel_dmeff': 0, 
    'dmeff_target': 'baryons', 
    'm_dmeff': 0.001
}
test_params_CDM = {
    'omega_b': 2.268118e-2, 
    'omega_cdm': 0.1221163, 
    'ln10^{10}A_s': 3.048006, 
    'n_s': 0.9670508, 
    'tau_reio': 0.05659527, 
    'Omega_Lambda': 0.6772064, 
    'YHe': 0.2479716, 
    'h': 67.13298e-2
}

In [ ]:
def run_class(selection):
    # run class on the user's selection
    cosmo = Class()
    cosmo.set(selection)
    cosmo.set({'output':'mPk, tCl, pCl, lCl','P_k_max_1/Mpc':3.0, 'lensing':'yes'})
    cosmo.compute()

    # set variables for matter power spectrum and lensed CMB angular power spectra
    kk = np.logspace(-4,np.log10(3),1000)
    Pk = []
    h = cosmo.h()
    for k in kk:
        Pk.append(cosmo.pk(k*h,0.)*h**3)
    Pk = np.array(Pk)
    l = np.array(range(2,2501))
    factor = l*(l+1)/(2*np.pi)
    lensed_cl = cosmo.lensed_cl(2500)
    
    results = {'k': kk, 'Pk': Pk, 'l': l, 'Cl_tt': factor*lensed_cl['tt'][2:], 'Cl_ee': factor*lensed_cl['ee'][2:]}
    return results

### Multiprocessing

Run the cells below and paste the output here, along with your system specifications.

#### System: Razer Blade 14 2021
- Processor: AMD Ryzen 9 5900HX with Radeon Graphics (3.30 GHz)
- Installed RAM: 16.0 GB
- On battery: 
    - **Multiprocessing: 5.2 s ± 164 ms**
    - **Serial processing: 9.41 s ± 481 ms**
- Plugged in: 
    - **Multiprocessing: 3.73 s ± 46.9 ms**
    - **Serial Processing: 6.76 s ± 80.9 ms**


#### System:
- Processor:
- Installed RAM: 
- On battery:
- Plugged in: 

In [ ]:
%%timeit
# --------------------------------- Multiprocessing ---------------------------------
if __name__ == '__main__':
    with Pool() as p:
        [mycosmo, LambdaCDM] = p.map(run_class, [test_params, test_params_CDM])

In [ ]:
%%timeit
# -------------------------------- Serial Processing --------------------------------
mycosmo = run_class(test_params)
LambdaCDM = run_class(test_params_CDM)